## ABCD method

### calculate cb,cc,cd

In [1]:
import awkward as ak
import uproot
import hist
import os
import numpy as np
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.nanoevents.methods.base import NanoEventsArray
from imp import reload #每次修改调用函数后可以更新
import correctionlib, rich
import matplotlib.pyplot as plt
import sys
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd

/tmp/ipykernel_329876/4119456318.py:8: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload #每次修改调用函数后可以更新


### 录入文件

In [29]:
def load_event_dict(file_prefix, keys):
    data_subkeys = ['photon_pt', 'photon_endcap','photon_barrel']#'photon_eta', 'muon1_pt', 'muon2_pt', 'muon1_eta', 'muon2_eta', 'muon_mass','gmumu_mass',
    other_subkeys = data_subkeys + ['generator_weight', 'event_weight']
    
    event_dict = {}
    for key in keys:
        subkeys = other_subkeys if 'data' not in key else data_subkeys
        event_dict[key] = {}
        for subkey in subkeys:
            event_dict[key][subkey] = ak.from_parquet(f'{file_prefix}_{key}_{subkey}.parquet')
    return event_dict

In [30]:
keys = ['dataC_muon','dataD_muon',"zg0_22","zg1_22", "zg2_22", "zg3_22","tt_22","zz_22", "ww_22", "wz_22","dy_22"]  
# 读取数据

event_A = load_event_dict('../selection_muon_file/v2/A/event_muon_final', keys)
event_B = load_event_dict('../selection_muon_file/v2/B/event_muon_final', keys)
event_C = load_event_dict('../selection_muon_file/v2/C/event_muon_final', keys)
event_D = load_event_dict('../selection_muon_file/v2/D/event_muon_final', keys)

In [9]:
keys = ['dataC_ele','dataD_ele',"zg0_22","zg1_22", "zg2_22", "zg3_22","tt_22","zz_22", "ww_22", "wz_22","dy_22"]  
# 读取数据 
event_A = load_event_dict('../selection_ele_file/v2/A/event_ele_final', keys)
event_B = load_event_dict('../selection_ele_file/v2/B/event_ele_final', keys)
event_C = load_event_dict('../selection_ele_file/v2/C/event_ele_final', keys)
event_D = load_event_dict('../selection_ele_file/v2/D/event_ele_final', keys)

### A

In [31]:
# 合并key
data_keys = ["dataC_muon", "dataD_muon"]
info_keys = list(event_A["dataD_muon"].keys())

# data_keys = ["dataC_ele", "dataD_ele"]
# info_keys = list(event_A["dataD_ele"].keys())

event_A["data"] = {}


# 遍历每个信息键
for info_key in info_keys:
    # 从 event_final 中提取 "data1", "data2", "data3", "data4" 这四个 key 的数据
    data_to_concatenate = [event_A[data_key][info_key] for data_key in data_keys]

    # 使用 ak.concatenate 将数据连接在一起
    event_A["data"][info_key] = ak.concatenate(data_to_concatenate)

del event_A["dataC_muon"]
del event_A["dataD_muon"]
# del event_A["dataC_ele"]
# del event_A["dataD_ele"]

# zg_keys = ["zg0_22","zg1_22", "zg2_22", "zg3_22"]
# event_A["zg"] = {}

# info_keys = list(event_A["zg0_22"].keys())

# # 遍历每个信息键
# for info_key in info_keys:
#     # 从 event_final 中提取 "zg1", "zg2", "zg3", "zg4" 这四个 key 的数据
#     zg_to_concatenate = [event_A[zg_key][info_key] for zg_key in zg_keys]

#     # 使用 ak.concatenate 将数据连接在一起
#     event_A["zg"][info_key] = ak.concatenate(zg_to_concatenate)


In [32]:

# 定义一个函数来计算endcap和barrel的事件数
def count_events(key, condition_key):
    return ak.sum(event_A[key][condition_key])
    # return ak.sum(ak.flatten(event_A[key][condition_key]))


keys = ["data","zg0_22","zg1_22", "zg2_22", "zg3_22","tt_22","zz_22", "ww_22", "wz_22","dy_22"]#, "zg0", "zg1", "zg2", "zg3", "tt", "zz", "wz", "ww","dy","zg"]
# 用于保存endcap和barrel的事件数的字典
endcap_counts = {}
barrel_counts = {}
print("Total events:")
for key in keys:
    print(f"{key}:", len(event_A[key]['photon_pt']))

print("\nFor endcap:")
for key in keys:
    count = count_events(key, 'photon_endcap')
    print(f"{key}:", count)
    endcap_counts[key] = count  # 保存到字典中

print("\nFor barrel:")
for key in keys:
    count = count_events(key, 'photon_barrel')
    print(f"{key}:", count)
    barrel_counts[key] = count  # 保存到字典中

# 将事件数保存到特定的变量中
for key in keys:
    vars()['A_' + key + '_endcap'] = endcap_counts[key]
    vars()['A_' + key + '_barrel'] = barrel_counts[key]


Total events:
data: 2030
zg0_22: 33590
zg1_22: 488728
zg2_22: 10329
zg3_22: 14825
tt_22: 777
zz_22: 48
ww_22: 59
wz_22: 204
dy_22: 1112

For endcap:
data: 426
zg0_22: 6673
zg1_22: 95912
zg2_22: 2096
zg3_22: 2700
tt_22: 118
zz_22: 6
ww_22: 12
wz_22: 52
dy_22: 241

For barrel:
data: 1599
zg0_22: 26815
zg1_22: 391170
zg2_22: 8197
zg3_22: 12075
tt_22: 655
zz_22: 42
ww_22: 46
wz_22: 152
dy_22: 867


### B

In [33]:
# 合并key
# data_keys = ["dataF", "dataG"]
event_B["data"] = {}

# info_keys = list(event_B["dataF"].keys())

# 遍历每个信息键
for info_key in info_keys:
    # 从 event_final 中提取 "data1", "data2", "data3", "data4" 这四个 key 的数据
    data_to_concatenate = [event_B[data_key][info_key] for data_key in data_keys]

    # 使用 ak.concatenate 将数据连接在一起
    event_B["data"][info_key] = ak.concatenate(data_to_concatenate)

# del event_B["dataC_muon"]
# del event_B["dataD_muon"]
# del event_B["dataC_ele"]
# del event_B["dataD_ele"]

In [34]:

# 定义一个函数来计算endcap和barrel的事件数
def count_events(key, condition_key):
    return ak.sum(event_B[key][condition_key])
    # return ak.sum(ak.flatten(event_B[key][condition_key]))

# keys = ["data", "zg0", "zg1", "zg2", "zg3", "tt", "zz", "wz", "ww","dy"]
# 用于保存endcap和barrel的事件数的字典
endcap_counts = {}
barrel_counts = {}
print("Total events:")
for key in keys:
    print(f"{key}:", len(event_B[key]['photon_pt']))

print("\nFor endcap:")
for key in keys:
    count = count_events(key, 'photon_endcap')
    print(f"{key}:", count)
    endcap_counts[key] = count  # 保存到字典中

print("\nFor barrel:")
for key in keys:
    count = count_events(key, 'photon_barrel')
    print(f"{key}:", count)
    barrel_counts[key] = count  # 保存到字典中

# 将事件数保存到特定的变量中
for key in keys:
    vars()['B_' + key + '_endcap'] = endcap_counts[key]
    vars()['B_' + key + '_barrel'] = barrel_counts[key]


Total events:
data: 1946
zg0_22: 8200
zg1_22: 131017
zg2_22: 2832
zg3_22: 4191
tt_22: 262
zz_22: 13
ww_22: 11
wz_22: 75
dy_22: 386

For endcap:
data: 246
zg0_22: 1658
zg1_22: 25822
zg2_22: 508
zg3_22: 744
tt_22: 43
zz_22: 4
ww_22: 3
wz_22: 10
dy_22: 64

For barrel:
data: 1691
zg0_22: 6521
zg1_22: 104771
zg2_22: 2318
zg3_22: 3433
tt_22: 218
zz_22: 9
ww_22: 8
wz_22: 65
dy_22: 322


### C

In [35]:
# 合并key
event_C["data"] = {}

# 遍历每个信息键
for info_key in info_keys:
    # 从 event_final 中提取 "data1", "data2", "data3", "data4" 这四个 key 的数据
    data_to_concatenate = [event_C[data_key][info_key] for data_key in data_keys]

    # 使用 ak.concatenate 将数据连接在一起
    event_C["data"][info_key] = ak.concatenate(data_to_concatenate)

# del event_C["dataC_muon"]
# del event_C["dataD_muon"]


In [36]:

# 定义一个函数来计算endcap和barrel的事件数
def count_events(key, condition_key):
    if len(event_C[key][condition_key]) != 0:
        return ak.sum(event_C[key][condition_key])
        # return ak.sum(ak.flatten(event_C[key][condition_key]))        
    else:
        return 0

# 用于保存endcap和barrel的事件数的字典
endcap_counts = {}
barrel_counts = {}
print("Total events:")
for key in keys:
    print(f"{key}:", len(event_C[key]['photon_pt']))

print("\nFor endcap:")
for key in keys:
    count = count_events(key, 'photon_endcap')
    print(f"{key}:", count)
    endcap_counts[key] = count  # 保存到字典中

print("\nFor barrel:")
for key in keys:
    count = count_events(key, 'photon_barrel')
    print(f"{key}:", count)
    barrel_counts[key] = count  # 保存到字典中

# 将事件数保存到特定的变量中
for key in keys:
    vars()['C_' + key + '_endcap'] = endcap_counts[key]
    vars()['C_' + key + '_barrel'] = barrel_counts[key]


Total events:
data: 28
zg0_22: 512
zg1_22: 1016
zg2_22: 2
zg3_22: 1
tt_22: 5
zz_22: 1
ww_22: 0
wz_22: 4
dy_22: 13

For endcap:
data: 16
zg0_22: 325
zg1_22: 740
zg2_22: 2
zg3_22: 0
tt_22: 2
zz_22: 1
ww_22: 0
wz_22: 4
dy_22: 9

For barrel:
data: 11
zg0_22: 183
zg1_22: 268
zg2_22: 0
zg3_22: 1
tt_22: 3
zz_22: 0
ww_22: 0
wz_22: 0
dy_22: 4


### D

In [37]:
# 合并key
event_D["data"] = {}

# 遍历每个信息键
for info_key in info_keys:
    # 从 event_final 中提取 "data1", "data2", "data3", "data4" 这四个 key 的数据
    data_to_concatenate = [event_D[data_key][info_key] for data_key in data_keys]

    # 使用 ak.concatenate 将数据连接在一起
    event_D["data"][info_key] = ak.concatenate(data_to_concatenate)

# del event_D["dataC_muon"]
# del event_D["dataD_muon"]


In [38]:

# 定义一个函数来计算endcap和barrel的事件数
def count_events(key, condition_key):
    if key not in event_D:
        return 0
    if condition_key not in event_D[key]:
        return 0
    if len(event_D[key][condition_key]) == 0:
        return 0
    return ak.sum(event_D[key][condition_key])
    # return ak.sum(ak.flatten(event_D[key][condition_key]))

# 用于保存endcap和barrel的事件数的字典
endcap_counts = {}
barrel_counts = {}
print("Total events:")
for key in keys:
    print(f"{key}:", len(event_D[key]['photon_pt']))

print("\nFor endcap:")
for key in keys:
    count = count_events(key, 'photon_endcap')
    print(f"{key}:", count)
    endcap_counts[key] = count  # 保存到字典中

print("\nFor barrel:")
for key in keys:
    count = count_events(key, 'photon_barrel')
    print(f"{key}:", count)
    barrel_counts[key] = count  # 保存到字典中

# 将事件数保存到特定的变量中
for key in keys:
    vars()['D_' + key + '_endcap'] = endcap_counts[key]
    vars()['D_' + key + '_barrel'] = barrel_counts[key]


Total events:
data: 74
zg0_22: 159
zg1_22: 366
zg2_22: 1
zg3_22: 1
tt_22: 2
zz_22: 0
ww_22: 1
wz_22: 1
dy_22: 6

For endcap:
data: 44
zg0_22: 100
zg1_22: 284
zg2_22: 1
zg3_22: 1
tt_22: 0
zz_22: 0
ww_22: 1
wz_22: 1
dy_22: 4

For barrel:
data: 30
zg0_22: 57
zg1_22: 77
zg2_22: 0
zg3_22: 0
tt_22: 2
zz_22: 0
ww_22: 0
wz_22: 0
dy_22: 2


# Method using NA_dataset

### DY & R

In [39]:
R_endcap = A_dy_22_endcap * D_dy_22_endcap / B_dy_22_endcap / C_dy_22_endcap
# D_dy_22_barrel = 1
R_barrel = A_dy_22_barrel * D_dy_22_barrel / B_dy_22_barrel / C_dy_22_barrel

print('R_endcap',R_endcap)
print('R_barrel',R_barrel)

R_endcap 1.6736111111111112
R_barrel 1.3462732919254659


In [40]:
1112 * XSEC["dy_22"] * LUMI['2022CD'] * 1e3 / ak.sum(np.sign(event_A["dy_22"]['event_weight']))

1263.8760148601127

### cc,cb,cd

In [43]:
LUMI = {"2022FG": 21.6,"2022EFG": 39.22,"2022EG": 9.0,"2022CD":7.87}
XSEC = {
    # unit is pb^-1
    'zg0_22': 124.6,
    'zg1_22': 1.74,#27.11
    'zg2_22': 0.30,#0.88,
    'zg3_22': 0.042,#0.073,
    'tt_22': 96.9,#101.78,
    'ww_22': 116.8,#80.27,
    'wz_22': 54.3,#29.14,
    'zz_22': 16.7,#12.83,
    'dy_22': 6346.0,
}


def compute_truenum(event, region, suffix, LUMI, XSEC, considered_keys):
    truenum_dict = {}
    for key in considered_keys:
        variable_name = f"{region}_{key}_{suffix}"  # 如 A_zg0_barrel
        event_count = globals()[variable_name]
        truenum_dict[key] = event_count * XSEC[key] * LUMI['2022CD'] * 1e3 / ak.sum(np.sign(event[key]['event_weight']))
    return truenum_dict

list0 = ['zg0_22','zg1_22','zg2_22','zg3_22']
list1 = ["tt_22", "ww_22", "wz_22", "zz_22"]


# considered_keys = [i for i in XSEC if (i != 'data') & (i != 'dy') & (i != 'zg')]
considered_keys = [i for i in XSEC]
# For A:
A_truenum_barrel = compute_truenum(event_A, "A", "barrel", LUMI, XSEC, considered_keys)
A_truenum_endcap = compute_truenum(event_A, "A", "endcap", LUMI, XSEC, considered_keys)

# A_prompt_barrel = sum(abs(A_truenum_barrel[key]) for key in considered_keys)
A_prompt_barrel = sum(abs(A_truenum_barrel[key]) for key in list0)
A_bkg_prompt_barrel = sum(abs(A_truenum_barrel[key]) for key in list1)
A_prompt_endcap = sum(abs(A_truenum_endcap[key]) for key in list0)
A_bkg_prompt_endcap = sum(abs(A_truenum_endcap[key]) for key in list1)

print('A_prompt_barrel', A_prompt_barrel)
print('A_bkg_prompt_barrel', A_bkg_prompt_barrel)
print('A_prompt_endcap', A_prompt_endcap)
print('A_bkg_prompt_endcap', A_bkg_prompt_endcap)

# For A:
B_truenum_barrel = compute_truenum(event_B, "B", "barrel", LUMI, XSEC, considered_keys)
B_truenum_endcap = compute_truenum(event_B, "B", "endcap", LUMI, XSEC, considered_keys)

B_prompt_barrel = sum(abs(B_truenum_barrel[key]) for key in list0)
B_bkg_prompt_barrel = sum(abs(B_truenum_barrel[key]) for key in list1)
B_prompt_endcap = sum(abs(B_truenum_endcap[key]) for key in list0)
B_bkg_prompt_endcap = sum(abs(B_truenum_endcap[key]) for key in list1)

print('B_prompt_barrel', B_prompt_barrel)
print('B_bkg_prompt_barrel', B_bkg_prompt_barrel)
print('B_prompt_endcap', B_prompt_endcap)
print('B_bkg_prompt_endcap', B_bkg_prompt_endcap)


# For A:
C_truenum_barrel = compute_truenum(event_C, "C", "barrel", LUMI, XSEC, considered_keys)
C_truenum_endcap = compute_truenum(event_C, "C", "endcap", LUMI, XSEC, considered_keys)

C_prompt_barrel = sum(abs(C_truenum_barrel[key]) for key in list0)
C_bkg_prompt_barrel = sum(abs(C_truenum_barrel[key]) for key in list1)
C_prompt_endcap = sum(abs(C_truenum_endcap[key]) for key in list0)
C_bkg_prompt_endcap = sum(abs(C_truenum_endcap[key]) for key in list1)

print('C_prompt_barrel', C_prompt_barrel)
print('C_bkg_prompt_barrel', C_bkg_prompt_barrel)
print('C_prompt_endcap', C_prompt_endcap)
print('C_bkg_prompt_endcap', C_bkg_prompt_endcap)

# For A:
D_truenum_barrel = compute_truenum(event_D, "D", "barrel", LUMI, XSEC, considered_keys)
D_truenum_endcap = compute_truenum(event_D, "D", "endcap", LUMI, XSEC, considered_keys)

D_prompt_barrel = sum(abs(D_truenum_barrel[key]) for key in list0)
D_bkg_prompt_barrel = sum(abs(D_truenum_barrel[key]) for key in list1)

D_prompt_endcap = sum(abs(D_truenum_endcap[key]) for key in list0)
D_bkg_prompt_endcap = sum(abs(D_truenum_endcap[key]) for key in list1)

print('D_prompt_barrel', D_prompt_barrel)
print('D_bkg_prompt_barrel', D_bkg_prompt_barrel)
print('D_prompt_endcap', D_prompt_endcap)
print('D_bkg_prompt_endcap', D_bkg_prompt_endcap)

A_prompt_barrel 2241.723553529841
A_bkg_prompt_barrel 37.23226176867323
A_prompt_endcap 555.7891952529843
A_bkg_prompt_endcap 8.15311812748058
B_prompt_barrel 568.7590895294487
B_bkg_prompt_barrel 12.218723887438783
B_prompt_endcap 141.23922403542588
B_bkg_prompt_endcap 2.5824938211550506
C_prompt_barrel 10.061034662591602
C_bkg_prompt_barrel 0.09699947273332568
C_prompt_endcap 18.325749257194545
C_bkg_prompt_endcap 0.4029782364014958
D_prompt_barrel 3.1224570880997398
D_bkg_prompt_barrel 0.06466631515555046
D_prompt_endcap 5.739681490291919
D_bkg_prompt_endcap 0.1166290501133194


In [44]:
print('A_bkg_prompt_barrel', A_bkg_prompt_barrel)
print('B_bkg_prompt_barrel', B_bkg_prompt_barrel)
print('C_bkg_prompt_barrel', C_bkg_prompt_barrel)
print('D_bkg_prompt_barrel', D_bkg_prompt_barrel)

print('A_bkg_prompt_endcap', A_bkg_prompt_endcap)
print('B_bkg_prompt_endcap', B_bkg_prompt_endcap)
print('C_bkg_prompt_endcap', C_bkg_prompt_endcap)
print('D_bkg_prompt_endcap', D_bkg_prompt_endcap)

A_bkg_prompt_barrel 37.23226176867323
B_bkg_prompt_barrel 12.218723887438783
C_bkg_prompt_barrel 0.09699947273332568
D_bkg_prompt_barrel 0.06466631515555046
A_bkg_prompt_endcap 8.15311812748058
B_bkg_prompt_endcap 2.5824938211550506
C_bkg_prompt_endcap 0.4029782364014958
D_bkg_prompt_endcap 0.1166290501133194


In [45]:
c_B_barrel = B_prompt_barrel/A_prompt_barrel
c_C_barrel = C_prompt_barrel/A_prompt_barrel
c_D_barrel = D_prompt_barrel/A_prompt_barrel

print('c_B_barrel', c_B_barrel)
print('c_C_barrel', c_C_barrel)
print('c_D_barrel', c_D_barrel)

c_B_endcap = B_prompt_endcap/A_prompt_endcap
c_C_endcap = C_prompt_endcap/A_prompt_endcap
c_D_endcap = D_prompt_endcap/A_prompt_endcap

print('c_B_endcap', c_B_endcap)
print('c_C_endcap', c_C_endcap)
print('c_D_endcap', c_D_endcap)

c_B_barrel 0.25371508838985735
c_C_barrel 0.00448808000734497
c_D_barrel 0.0013928823128896007
c_B_endcap 0.25412373115878323
c_C_endcap 0.03297248203764203
c_D_endcap 0.010327083612482479


## data

In [46]:
print('A_data_barrel',A_data_barrel)
print('B_data_barrel',B_data_barrel)
print('C_data_barrel',C_data_barrel)
print('D_data_barrel',D_data_barrel)
print('A_data_endcap',A_data_endcap)
print('B_data_endcap',B_data_endcap)
print('C_data_endcap',C_data_endcap)
print('D_data_endcap',D_data_endcap)

A_data_barrel 1599
B_data_barrel 1691
C_data_barrel 11
D_data_barrel 30
A_data_endcap 426
B_data_endcap 246
C_data_endcap 16
D_data_endcap 44


### calculate NA(nonprompt)

In [53]:
### ----------------------  MUON --------

import sympy as sp

# 定义变量
NA_prompt, NA, NA_bkg, R, NB, cB, NA_prompt, NB_bkg, NC, cC, ND, cD, NC_bkg, ND_bkg = sp.symbols('NA_prompt NA NA_bkg R NB cB NA_prompt NB_bkg NC cC ND cD NC_bkg ND_bkg')

# 根据公式【1】定义方程
equation = sp.Eq(NA_prompt, NA - NA_bkg - R * (NB - cB * NA_prompt - NB_bkg) * (NC - cC * NA_prompt - NC_bkg) / (ND - cD * NA_prompt - ND_bkg))


# 已知的值
values_barrel = {
    NA: 1599,  # A_data
    R: 1.3462732919254659,  # 
    NB: 1691,  # B_data
    cB: 0.25371508838985735,  # 
    NC: 11,  # C_data
    cC: 0.00448808000734497,  # 
    ND: 30,  # D_data
    cD: 0.0013928823128896007,  # 
    NA_bkg: 37.23226176867323,
    NB_bkg: 12.218723887438783,  
    NC_bkg: 0.09699947273332568,  
    ND_bkg: 0.06466631515555046,  
}


# 解方程
solution = sp.solve(equation.subs(values_barrel), NA_prompt)

# 输出解
print(f"NA_prompt_barrel = {solution[0]}")
print(f"NA_prompt_barrel = {solution[1]}")
NA_prompt_barrel = solution[1]


values_endcap = {
    NA: 426,  # A_data
    R: 1.6736111111111112,  
    NB: 246,  # B_data
    cB: 0.25412373115878323,  # 
    NC: 16,  # C_data
    cC: 0.03297248203764203,  # 
    ND: 44,  # D_data
    cD: 0.010327083612482479,  # 
    NA_bkg: 8.15311812748058,
    NB_bkg: 2.5824938211550506,  # 
    NC_bkg: 0.4029782364014958,  # 
    ND_bkg: 0.1166290501133194,  # 
}
# 解方程
solution = sp.solve(equation.subs(values_endcap), NA_prompt)

# 输出解
print(f"NA_prompt_endcap = {solution[0]}")
print(f"NA_prompt_endcap = {solution[1]}")
NA_prompt_endcap = solution[1]



NA_prompt_barrel = -131405.669350307
NA_prompt_barrel = 1200.90211771034
NA_prompt_endcap = -8015.52130615924
NA_prompt_endcap = 404.440281521520


In [54]:
N_nonprompt_barrel = A_data_barrel - A_bkg_prompt_barrel - NA_prompt_barrel
N_nonprompt_endcap = A_data_endcap - A_bkg_prompt_endcap - NA_prompt_endcap

print('N_nonprompt_barrel = ',N_nonprompt_barrel)
print('N_nonprompt_endcap = ',N_nonprompt_endcap)

N_nonprompt_barrel =  360.865620520985
N_nonprompt_endcap =  13.4066003509992


In [55]:
### ----------------------  ELE --------
import sympy as sp

# 定义变量
NA_prompt, NA, NA_bkg, R, NB, cB, NA_prompt, NB_bkg, NC, cC, ND, cD, NC_bkg, ND_bkg = sp.symbols('NA_prompt NA NA_bkg R NB cB NA_prompt NB_bkg NC cC ND cD NC_bkg ND_bkg')

# 根据公式【1】定义方程
equation = sp.Eq(NA_prompt, NA - NA_bkg - R * (NB - cB * NA_prompt - NB_bkg) * (NC - cC * NA_prompt - NC_bkg) / (ND - cD * NA_prompt - ND_bkg))

# 已知的值
values_barrel = {
    NA: 1284,  # A_data 
    R: 1,  # 0
    NB: 1234,  # B_data
    cB: 1.8217523765018677,  # 
    NC: 12,  # C_data
    cC: 0.06640036970456702,  # 
    ND: 21,  # D_data
    cD: 0.012617905913563331,  # 
    NA_bkg: 28.169547659446664,
    NB_bkg: 8.650767050388101,  
    NC_bkg: 0.4383922685341407,  
    ND_bkg: 0.11121557012904591,  
}


# 解方程
solution = sp.solve(equation.subs(values_barrel), NA_prompt)

# 输出解
print(f"NA_prompt_barrel = {solution[0]}")
print(f"NA_prompt_barrel = {solution[1]}")
NA_prompt_barrel = solution[1]


values_endcap = {
    NA: 259,  # A_data
    R: 1,  #10.227272727272727
    NB: 186,  # B_data
    cB: 1.4595056891498097,  # 
    NC: 11,  # C_data
    cC: 0.19714020497038526,  # 
    ND: 37,  # D_data
    cD: 0.09450407437661776,  # 
    NA_bkg: 5.0185361399050725,
    NB_bkg: 1.809650732949314,  # 
    NC_bkg: 0.2026554209154498,  # 
    ND_bkg: 0.0891072453570001,  # 
}
# 解方程
solution = sp.solve(equation.subs(values_endcap), NA_prompt)

# 输出解
print(f"NA_prompt_endcap = {solution[0]}")
print(f"NA_prompt_endcap = {solution[1]}")
NA_prompt_endcap = solution[1]



NA_prompt_barrel = -147.695254888733
NA_prompt_barrel = 753.999236511238
NA_prompt_endcap = -219.729238576951
NA_prompt_endcap = 173.963161110441


In [27]:
N_nonprompt_barrel = A_data_barrel - A_bkg_prompt_barrel - NA_prompt_barrel
N_nonprompt_endcap = A_data_endcap - A_bkg_prompt_endcap - NA_prompt_endcap

print('N_nonprompt_barrel = ',N_nonprompt_barrel)
print('N_nonprompt_endcap = ',N_nonprompt_endcap)

N_nonprompt_barrel =  501.831215829316
N_nonprompt_endcap =  80.0183027496542


In [24]:
import sympy as sp

# 定义变量
NA_prompt, NA, NA_bkg, R, NB, cB, NC, cC, ND, cD, NB_bkg, NC_bkg, ND_bkg = sp.symbols('NA_prompt NA NA_bkg R NB cB NC cC ND cD NB_bkg NC_bkg ND_bkg')

# 根据公式【1】定义方程
equation = sp.Eq(NA_prompt, NA - NA_bkg - R * (NB - cB * NA_prompt - NB_bkg) * (NC - cC * NA_prompt - NC_bkg) / (ND - cD * NA_prompt - ND_bkg))

# # 已知的值
# values = {
#     NA: 4103,  # A_data
#     R: 1.157,  # 
#     NB: 4007,  # B_data
#     cB: 0.268,  # 
#     NC: 107,  # C_data
#     cC: 0.0035,  # 
#     ND: 279,  # D_data
#     cD: 0.0013,  # 
#     NA_bkg: 263.0754951199856,
#     NB_bkg: 64.41697392770189,  # B_dy
#     NC_bkg: 3.5464861774302974,  # C_dy
#     ND_bkg: 1.1569442372019945,  # D_dy
# }

# 已知的值
values_barrel = {
    NA: 2621,  # A_data
    R: 1.3210,  # 
    NB: 2415,  # B_data
    cB: 0.28579264766901313,  # 
    NC: 20,  # C_data
    cC: 0.007219130190589022,  # 
    ND: 34,  # D_data
    cD: 0.0008892123680511543,  # 
    NA_bkg: 182.88692,
    NB_bkg: 45.410448625775594,  
    NC_bkg: 0.7525415693858519,  
    ND_bkg: 0.18241031668551638,  
}

values_endcap = {
    NA: 1471,  # A_data
    R: 1.177103,  
    NB: 1574,  # B_data
    cB: 0.2735580021848148,  # 
    NC: 87,  # C_data
    cC: 0.024829216269293436,  # 
    ND: 245,  # D_data
    cD: 0.013236878607708733,  # 
    NA_bkg: 78.75595088454901,
    NB_bkg: 18.845132891691144,  # B_dy
    NC_bkg: 2.793944608044445,  # C_dy
    ND_bkg: 0.9745339205164781,  # D_dy
}

# 解方程得到NA_prompt的值
NA_prompt_value = sp.solve(equation.subs(values_barrel), NA_prompt)[0]

# 计算公式【1】中每个变量的偏导数
partial_derivatives = {
    var: equation.lhs.diff(var) for var in [R, NB, cB, NC, cC, ND, cD]
}

# 计算每个变量的不确定度
uncertainties = {
    R: 0.2845 * values_barrel[R],
    NB: sp.sqrt(values_barrel[NB]),
    cB: sp.sqrt((sp.sqrt(NB)/NB)**2 + (sp.sqrt(NA)/NA)**2).subs(values_barrel),
    NC: sp.sqrt(values_barrel[NC]),
    cC: sp.sqrt((sp.sqrt(NC)/NC)**2 + (sp.sqrt(NC)/NC)**2).subs(values_barrel),
    ND: sp.sqrt(values_barrel[ND]),
    cD: sp.sqrt((sp.sqrt(ND)/ND)**2 + (sp.sqrt(ND)/ND)**2).subs(values_barrel),
}

# 使用误差传递公式计算NA_prompt的不确定度
sigma_NA_prompt_squared = sum([
    (partial_derivatives[var].subs(NA_prompt, NA_prompt_value) * uncertainties[var])**2 for var in partial_derivatives
])

sigma_NA_prompt = sp.sqrt(sigma_NA_prompt_squared)

# 使用公式【2】计算N_nonprompt的不确定度
sigma_N_nonprompt = sp.sqrt(values_barrel[NA] + values_barrel[NA_bkg] + sigma_NA_prompt**2)

print(f"NA_prompt uncertainty = {sigma_NA_prompt}")
print(f"N_nonprompt uncertainty = {sigma_N_nonprompt}")

# 解方程
solution = sp.solve(equation.subs(values_barrel), NA_prompt)

# 输出解
# print(f"NA_prompt = {solution[0]}")
# print(f"NA_prompt = {solution[1]}")
NA_prompt = solution[1]
N_nonprompt = A_data_barrel - A_bkg_prompt_barrel - NA_prompt
print("NA_nonprompt =",N_nonprompt)


NA_prompt uncertainty = 0
N_nonprompt uncertainty = 52.9517414255660
NA_nonprompt = 1112.24095050912


In [27]:
import sympy as sp

# 定义变量
NA_prompt, NA, NA_bkg, R, NB, cB, NC, cC, ND, cD, NB_bkg, NC_bkg, ND_bkg = sp.symbols('NA_prompt NA NA_bkg R NB cB NC cC ND cD NB_bkg NC_bkg ND_bkg')

# 根据公式【1】定义方程
equation = sp.Eq(NA_prompt, NA - NA_bkg - R * (NB - cB * NA_prompt - NB_bkg) * (NC - cC * NA_prompt - NC_bkg) / (ND - cD * NA_prompt - ND_bkg))


values_endcap = {
    NA: 1471,  # A_data
    R: 1.177103,  
    NB: 1574,  # B_data
    cB: 0.2668905,  # 
    NC: 87,  # C_data
    cC: 0.0319460,  # 
    ND: 245,  # D_data
    cD: 0.012202605,  # 
    NA_bkg: 78.75595088454901,
    NB_bkg: 18.845132891691144,  # B_dy
    NC_bkg: 2.793944608044445,  # C_dy
    ND_bkg: 0.9745339205164781,  # D_dy
}

# 解方程得到NA_prompt的值
NA_prompt_value = sp.solve(equation.subs(values_endcap), NA_prompt)[0]

# 计算公式【1】中每个变量的偏导数
partial_derivatives = {
    var: equation.lhs.diff(var) for var in [R, NB, cB, NC, cC, ND, cD]
}

# 计算每个变量的不确定度
uncertainties = {
    R: 0.2845 * values_endcap[R],
    NB: sp.sqrt(values_endcap[NB]),
    cB: sp.sqrt((sp.sqrt(NB)/NB)**2 + (sp.sqrt(NA)/NA)**2).subs(values_endcap),
    NC: sp.sqrt(values_endcap[NC]),
    cC: sp.sqrt((sp.sqrt(NC)/NC)**2 + (sp.sqrt(NC)/NC)**2).subs(values_endcap),
    ND: sp.sqrt(values_endcap[ND]),
    cD: sp.sqrt((sp.sqrt(ND)/ND)**2 + (sp.sqrt(ND)/ND)**2).subs(values_endcap),
}

# 使用误差传递公式计算NA_prompt的不确定度
sigma_NA_prompt_squared = sum([
    (partial_derivatives[var].subs(NA_prompt, NA_prompt_value) * uncertainties[var])**2 for var in partial_derivatives
])

sigma_NA_prompt = sp.sqrt(sigma_NA_prompt_squared)

# 使用公式【2】计算N_nonprompt的不确定度
sigma_N_nonprompt = sp.sqrt(values_endcap[NA] + values_endcap[NA_bkg] + sigma_NA_prompt**2)

print(f"NA_prompt uncertainty = {sigma_NA_prompt}")
print(f"N_nonprompt uncertainty = {sigma_N_nonprompt}")

# 解方程
solution = sp.solve(equation.subs(values_endcap), NA_prompt)

# 输出解
NA_prompt = solution[0]
N_nonprompt = A_data_endcap - A_bkg_prompt_endcap - NA_prompt
print("NA_nonprompt =",N_nonprompt)


NA_prompt uncertainty = 0
N_nonprompt uncertainty = 39.3669398211818
NA_nonprompt = 324.161701802574


### 老结果副本

In [ ]:
import sympy as sp

# 定义变量
NA_prompt, NA, NA_bkg, R, NB, cB, NA_prompt, NB_bkg, NC, cC, ND, cD, NC_bkg, ND_bkg = sp.symbols('NA_prompt NA NA_bkg R NB cB NA_prompt NB_bkg NC cC ND cD NC_bkg ND_bkg')

# 根据公式【1】定义方程
equation = sp.Eq(NA_prompt, NA - NA_bkg - R * (NB - cB * NA_prompt - NB_bkg) * (NC - cC * NA_prompt - NC_bkg) / (ND - cD * NA_prompt - ND_bkg))

# 已知的值
values_barrel = {
    NA: 2621,  # A_data
    R: 1.3210,  # 
    NB: 2415,  # B_data
    cB: 0.264648770,  # 
    NC: 20,  # C_data
    cC: 0.0043251,  # 
    ND: 34,  # D_data
    cD: 0.0010703,  # 
    NA_bkg: 182.88692,
    NB_bkg: 45.410448625775594,  
    NC_bkg: 0.7525415693858519,  
    ND_bkg: 0.18241031668551638,  
}


# 解方程
solution = sp.solve(equation.subs(values_barrel), NA_prompt)

# 输出解
print(f"NA_prompt_barrel = {solution[0]}")
print(f"NA_prompt_barrel = {solution[1]}")
NA_prompt_barrel = solution[1]

values_endcap = {
    NA: 1471,  # A_data
    R: 1.177103,  
    NB: 1574,  # B_data
    cB: 0.2668905,  # 
    NC: 87,  # C_data
    cC: 0.0319460,  # 
    ND: 245,  # D_data
    cD: 0.012202605,  # 
    NA_bkg: 78.75595088454901,
    NB_bkg: 18.845132891691144,  # B_dy
    NC_bkg: 2.793944608044445,  # C_dy
    ND_bkg: 0.9745339205164781,  # D_dy
}
# 解方程
solution = sp.solve(equation.subs(values_endcap), NA_prompt)

# 输出解
print(f"NA_prompt_endcap = {solution[0]}")
print(f"NA_prompt_endcap = {solution[1]}")
NA_prompt_endcap = solution[0]



## 老版本：整体

### A

In [ ]:
# 合并key
data_keys = ["dataF", "dataG"]
event_A["data_endcap"] = {}
event_A["data_barrel"] = {}

info_keys = list(event_A["dataF"].keys())

# 遍历每个信息键
for info_key in info_keys:
    # 从 event_final 中提取 "data1", "data2", "data3", "data4" 这四个 key 的数据
    data_to_concatenate = [event_A[data_key][info_key] for data_key in data_keys]

    # 使用 ak.concatenate 将数据连接在一起
    event_A["data"][info_key] = ak.concatenate(data_to_concatenate)

# data_keys = ["zg0", "zg1","zg2","zg3"]
# event_A["zg"] = {}

# info_keys = list(event_A["zg0"].keys())

# # 遍历每个信息键
# for info_key in info_keys:
#     # 从 event_final 中提取 "data1", "data2", "data3", "data4" 这四个 key 的数据
#     data_to_concatenate = [event_A[data_key][info_key] for data_key in data_keys]

#     # 使用 ak.concatenate 将数据连接在一起
#     event_A["zg"][info_key] = ak.concatenate(data_to_concatenate)

del event_A["dataF"]
del event_A["dataG"]
# del event_A["zg0"]
# del event_A["zg1"]
# del event_A["zg2"]
# del event_A["zg3"]
print('A_data',len(event_A['data']['photon_pt']))
print('A_zg0',len(event_A['zg0']['photon_pt']))
print('A_zg1',len(event_A['zg1']['photon_pt']))
print('A_zg2',len(event_A['zg2']['photon_pt']))
print('A_zg3',len(event_A['zg3']['photon_pt']))
print('A_tt',len(event_A['tt']['photon_pt']))
print('A_zz',len(event_A['zz']['photon_pt']))
print('A_wz',len(event_A['wz']['photon_pt']))
print('A_ww',len(event_A['ww']['photon_pt']))

A_data 4103
A_zg0 92130
A_zg1 1081280
A_zg2 27236
A_zg3 30551
A_tt 1430
A_zz 197
A_wz 4660
A_ww 184


### B

In [ ]:
keys = ['dataF', 'dataG', 'zg0', 'zg1', 'zg2','zg3','dy','tt','zz','wz','ww']  
# 读取数据
event_B = load_event_dict('ABCD_file/B/event_final', keys)
# event_B = load_event_dict('test_file/B/event_final', keys)

In [ ]:
# 合并key
data_keys = ["dataF", "dataG"]
event_B["data"] = {}

info_keys = list(event_B["dataF"].keys())

# 遍历每个信息键
for info_key in info_keys:
    # 从 event_final 中提取 "data1", "data2", "data3", "data4" 这四个 key 的数据
    data_to_concatenate = [event_B[data_key][info_key] for data_key in data_keys]

    # 使用 ak.concatenate 将数据连接在一起
    event_B["data"][info_key] = ak.concatenate(data_to_concatenate)

# 将 "data1", "data2", "data3", "data4" 这四个 key 的内容合并为 "data" 这个 key
data_keys = ["zg0", "zg1","zg2","zg3"]
event_B["zg"] = {}

info_keys = list(event_B["zg0"].keys())

# 遍历每个信息键
for info_key in info_keys:
    # 从 event_final 中提取 "data1", "data2", "data3", "data4" 这四个 key 的数据
    data_to_concatenate = [event_B[data_key][info_key] for data_key in data_keys]

    # 使用 ak.concatenate 将数据连接在一起
    event_B["zg"][info_key] = ak.concatenate(data_to_concatenate)

del event_B["dataF"]
del event_B["dataG"]
# del event_B["zg0"]
# del event_B["zg1"]
# del event_B["zg2"]
# del event_B["zg3"]
print('B_data',len(event_B['data']['photon_pt']))
print('B_zg0',len(event_B['zg0']['photon_pt']))
print('B_zg1',len(event_B['zg1']['photon_pt']))
print('B_zg2',len(event_B['zg2']['photon_pt']))
print('B_zg3',len(event_B['zg3']['photon_pt']))
print('B_tt',len(event_B['tt']['photon_pt']))
print('B_zz',len(event_B['zz']['photon_pt']))
print('B_wz',len(event_B['wz']['photon_pt']))
print('B_ww',len(event_B['ww']['photon_pt']))

B_data 4007
B_zg0 23006
B_zg1 260764
B_zg2 7485
B_zg3 8461
B_tt 492
B_zz 59
B_wz 1038
B_ww 42


### C

In [ ]:
keys = ['dataF', 'dataG', 'zg0', 'zg1', 'zg2','zg3','dy','tt','zz','wz','ww'] 
# 读取数据
event_C = load_event_dict('ABCD_file/C/event_final', keys)
# event_C = load_event_dict('test_file/C/event_final', keys)

In [ ]:
# 合并key
data_keys = ["dataF", "dataG"]
event_C["data"] = {}

info_keys = list(event_C["dataF"].keys())

# 遍历每个信息键
for info_key in info_keys:
    # 从 event_final 中提取 "data1", "data2", "data3", "data4" 这四个 key 的数据
    data_to_concatenate = [event_C[data_key][info_key] for data_key in data_keys]

    # 使用 ak.concatenate 将数据连接在一起
    event_C["data"][info_key] = ak.concatenate(data_to_concatenate)

#将 "data1", "data2", "data3", "data4" 这四个 key 的内容合并为 "data" 这个 key
data_keys = ["zg0", "zg1","zg2","zg3"]
event_C["zg"] = {}

info_keys = list(event_C["zg0"].keys())

# 遍历每个信息键
for info_key in info_keys:
    # 从 event_final 中提取 "data1", "data2", "data3", "data4" 这四个 key 的数据
    data_to_concatenate = [event_C[data_key][info_key] for data_key in data_keys]

    # 使用 ak.concatenate 将数据连接在一起
    event_C["zg"][info_key] = ak.concatenate(data_to_concatenate)

del event_C["dataF"]
del event_C["dataG"]
# del event_C["zg0"]
# del event_C["zg1"]
# del event_C["zg2"]
# del event_C["zg3"]
print('C_data',len(event_C['data']['photon_pt']))
# print('C_zg',len(event_C['zg']['photon_pt']))
print('C_zg0',len(event_C['zg0']['photon_pt']))
print('C_zg1',len(event_C['zg1']['photon_pt']))
print('C_zg2',len(event_C['zg2']['photon_pt']))
print('C_zg3',len(event_C['zg3']['photon_pt']))
print('C_tt',len(event_C['tt']['photon_pt']))
print('C_zz',len(event_C['zz']['photon_pt']))
print('C_wz',len(event_C['wz']['photon_pt']))
print('C_ww',len(event_C['ww']['photon_pt']))

C_data 107
C_zg0 1586
C_zg1 2273
C_zg2 1
C_zg3 6
C_tt 20
C_zz 4
C_wz 60
C_ww 2


### D

In [ ]:
keys = ['dataF', 'dataG', 'zg0', 'zg1', 'zg2','zg3','dy','tt','zz','wz','ww'] 
# 读取数据
event_D = load_event_dict('ABCD_file/D/event_final', keys)
# event_D = load_event_dict('test_file/D/event_final', keys)

In [ ]:
# 合并key
data_keys = ["dataF", "dataG"]
event_D["data"] = {}

info_keys = list(event_D["dataF"].keys())

# 遍历每个信息键
for info_key in info_keys:
    # 从 event_final 中提取 "data1", "data2", "data3", "data4" 这四个 key 的数据
    data_to_concatenate = [event_D[data_key][info_key] for data_key in data_keys]

    # 使用 ak.concatenate 将数据连接在一起
    event_D["data"][info_key] = ak.concatenate(data_to_concatenate)

# 将 "data1", "data2", "data3", "data4" 这四个 key 的内容合并为 "data" 这个 key
data_keys = ["zg0", "zg1","zg2","zg3"]
event_D["zg"] = {}

info_keys = list(event_D["zg0"].keys())

# 遍历每个信息键
for info_key in info_keys:
    # 从 event_final 中提取 "data1", "data2", "data3", "data4" 这四个 key 的数据
    data_to_concatenate = [event_D[data_key][info_key] for data_key in data_keys]

    # 使用 ak.concatenate 将数据连接在一起
    event_D["zg"][info_key] = ak.concatenate(data_to_concatenate)

del event_D["dataF"]
del event_D["dataG"]
# del event_D["zg0"]
# del event_D["zg1"]
# del event_D["zg2"]
# del event_D["zg3"]
print('D_data',len(event_D['data']['photon_pt']))
# print('D_zg',len(event_D['zg']['photon_pt']))
print('D_zg0',len(event_D['zg0']['photon_pt']))
print('D_zg1',len(event_D['zg1']['photon_pt']))
print('D_zg2',len(event_D['zg2']['photon_pt']))
print('D_zg3',len(event_D['zg3']['photon_pt']))
print('D_tt',len(event_D['tt']['photon_pt']))
print('D_zz',len(event_D['zz']['photon_pt']))
print('D_wz',len(event_D['wz']['photon_pt']))
print('D_ww',len(event_D['ww']['photon_pt']))

D_data 279
D_zg0 548
D_zg1 863
D_zg2 2
D_zg3 0
D_tt 11
D_zz 2
D_wz 14
D_ww 2


### data

In [ ]:
A_data = len(event_A['data']['photon_pt'])
B_data = len(event_B['data']['photon_pt'])
C_data = len(event_C['data']['photon_pt'])
D_data = len(event_D['data']['photon_pt'])
print('A_data',A_data)
print('B_data',B_data)
print('C_data',C_data)
print('D_data',D_data)

A_data 4103
B_data 4007
C_data 107
D_data 279


# Method using MC only

## zg

In [ ]:
LUMI = {"2016pre": 19.52, "2016post": 16.81, "2017": 41.48, "2018": 59.83, "2022CD": 6.36,"2022FG": 21.6,"2022EFG": 39.22}
XSEC = {
    # unit is pb^-1
    'zg0': 124.6,
    'zg1': 1.74,#27.11
    'zg2': 0.30,#0.88,
    'zg3': 0.042,#0.073,
    'tt': 96.9,#101.78,
    'ww': 116.8,#80.27,
    'wz': 54.3,#29.14,
    'zz': 16.7,#12.83,
}
A_truenum = {
    i: (
        (
            len(event_A[i]['photon_pt']) * XSEC[i] * LUMI['2022FG'] * 1e3 / ak.sum(np.sign(event_A[i]['event_weight']))
        )
    ) for i in event_A if (i != 'data') & (i!='dy') & (i!='zg')
}
A_prompt = abs(A_truenum['zg0'])+abs(A_truenum['zg1'])+abs(A_truenum['zg2'])+abs(A_truenum['zg3'])+abs(A_truenum['tt'])+abs(A_truenum['ww'])+abs(A_truenum['wz'])+abs(A_truenum['zz'])
A_bkg_prompt = abs(A_truenum['tt'])+abs(A_truenum['ww'])+abs(A_truenum['wz'])+abs(A_truenum['zz'])
print('A_prompt',A_prompt)
print('A_bkg_prompt',A_bkg_prompt)
B_truenum = {
    i: (
        (
            len(event_B[i]['photon_pt']) * XSEC[i] * LUMI['2022FG'] * 1e3 / ak.sum(np.sign(event_B[i]['event_weight']))
        )
    ) for i in event_B if (i != 'data') & (i!='dy') & (i!='zg')
}
B_prompt = abs(B_truenum['zg0'])+abs(B_truenum['zg1'])+abs(B_truenum['zg2'])+abs(B_truenum['zg3'])+abs(B_truenum['tt'])+abs(B_truenum['ww'])+abs(B_truenum['wz'])+abs(B_truenum['zz'])
B_bkg_prompt = abs(B_truenum['tt'])+abs(B_truenum['ww'])+abs(B_truenum['wz'])+abs(B_truenum['zz'])
print('B_prompt',B_prompt)
print('B_bkg_prompt',B_bkg_prompt)
C_truenum = {
    i: (
        (
            len(event_C[i]['photon_pt']) * XSEC[i] * LUMI['2022FG'] * 1e3 / ak.sum(np.sign(event_C[i]['event_weight']))
        )
    ) for i in event_C if (i != 'data') & (i!='dy') & (i!='zg')
}
C_prompt = abs(C_truenum['zg0'])+abs(C_truenum['zg1'])+abs(C_truenum['zg2'])+abs(C_truenum['zg3'])+abs(C_truenum['tt'])+abs(C_truenum['ww'])+abs(C_truenum['wz'])+abs(C_truenum['zz'])
C_bkg_prompt = abs(C_truenum['tt'])+abs(C_truenum['ww'])+abs(C_truenum['wz'])+abs(C_truenum['zz'])
print('C_prompt',C_prompt)
print('C_bkg_prompt',C_bkg_prompt)
D_truenum = {
    i: (
        (
            len(event_D[i]['photon_pt']) * XSEC[i] * LUMI['2022FG'] * 1e3 / ak.sum(np.sign(event_D[i]['event_weight']))
        )
    ) for i in event_D if (i != 'data') & (i!='dy') & (i!='zg')
}
D_prompt = abs(D_truenum['zg0'])+abs(D_truenum['zg1'])+abs(D_truenum['zg2'])+abs(D_truenum['zg3'])+abs(D_truenum['tt'])+abs(D_truenum['ww'])+abs(D_truenum['wz'])+abs(D_truenum['zz'])
D_bkg_prompt = abs(D_truenum['tt'])+abs(D_truenum['ww'])+abs(D_truenum['wz'])+abs(D_truenum['zz'])
print('D_prompt',D_prompt)
print('D_bkg_prompt',D_bkg_prompt)

A_prompt 6481.6380519282875
A_bkg_prompt 263.0754951199856
B_prompt 1664.4474675365414
B_bkg_prompt 64.41697392770189
C_prompt 75.9491372036926
C_bkg_prompt 3.5464861774302974
D_prompt 26.332492584624983
D_bkg_prompt 1.1569442372019945


In [ ]:
A_prompt=6481.6380519282875
A_data_2 = 4103-N_A_nonprompt
A_data_2

3769.814201140015

## calculation

In [ ]:
# R=1
R = 1.157
# N_A_nonprompt = R * (B_data-B_prompt/6481.638*3122.913)*(C_data-C_prompt/6481.638*3122.913)/(D_data-D_prompt/6481.638*3122.913)
N_A_nonprompt = R * (B_data-B_prompt)*(C_data-C_prompt)/(D_data-D_prompt)
N_A_nonprompt

333.0787867118313

In [ ]:
import numpy as np

# Given values
N_B = 4007
N_C = 107
N_D = 279
N_B_prompt = 1664.447
N_C_prompt = 75.949
N_D_prompt = 26.332

N_A_DY = 2520
N_B_DY = 673
N_C_DY = 55
N_D_DY = 17

# Calculate R and its uncertainty
R = (N_A_DY * N_D_DY) / (N_B_DY * N_C_DY)
sigma_N_A_DY = np.sqrt(N_A_DY)
sigma_N_B_DY = np.sqrt(N_B_DY)
sigma_N_C_DY = np.sqrt(N_C_DY)
sigma_N_D_DY = np.sqrt(N_D_DY)
sigma_R = R * np.sqrt((sigma_N_A_DY/N_A_DY)**2 + (sigma_N_B_DY/N_B_DY)**2 + (sigma_N_C_DY/N_C_DY)**2 + (sigma_N_D_DY/N_D_DY)**2)

# Calculate N_A_nonprompt and its uncertainty
N_A_nonprompt = R * (N_B - N_B_prompt) * (N_C - N_C_prompt) / (N_D - N_D_prompt)
sigma_N_B = np.sqrt(N_B)
sigma_N_C = np.sqrt(N_C)
sigma_N_D = np.sqrt(N_D)
sigma_N_A_nonprompt = N_A_nonprompt * np.sqrt((sigma_R/R)**2 + (sigma_N_B/N_B)**2 + (sigma_N_C/N_C)**2 + (sigma_N_D/N_D)**2)

print(f"R = {R:.2f} ± {sigma_R:.2f}")
print(f"N_A_nonprompt = {N_A_nonprompt:.2f} ± {sigma_N_A_nonprompt:.2f}")


R = 1.16 ± 0.33
N_A_nonprompt = 333.19 ± 101.10


# Method using NA_dataset

### DY & R

In [ ]:
A_dy = len(event_A['dy']['photon_pt'])
B_dy = len(event_B['dy']['photon_pt'])
C_dy = len(event_C['dy']['photon_pt'])
D_dy = len(event_D['dy']['photon_pt'])
R = A_dy * D_dy / B_dy / C_dy
print('A_dy',A_dy)
print('B_dy',B_dy)
print('C_dy',C_dy)
print('D_dy',D_dy)
print('A/C',A_dy/C_dy)
print('B/D',B_dy/D_dy)


print('R',R)

A_dy 2520
B_dy 673
C_dy 55
D_dy 17
A/C 45.81818181818182
B/D 39.588235294117645
R 1.1573686343374308


### cc,cb,cd

In [ ]:
c_B = len(event_B['zg']['photon_pt'])/len(event_A['zg']['photon_pt'])
c_C = len(event_C['zg']['photon_pt'])/len(event_A['zg']['photon_pt'])
c_D = len(event_D['zg']['photon_pt'])/len(event_A['zg']['photon_pt'])
print('A_zg',len(event_A['zg']['photon_pt']))
print('B_zg',len(event_B['zg']['photon_pt']))
print('C_zg',len(event_C['zg']['photon_pt']))
print('D_zg',len(event_D['zg']['photon_pt']))
print('c_B', c_B)
print('c_C', c_C)
print('c_D', c_D)

A_zg 1119858
B_zg 299716
C_zg 3866
D_zg 1413
c_B 0.26763750404069087
c_C 0.0034522234068962315
c_D 0.0012617671169023216


## data

In [ ]:
A_data = len(event_A['data']['photon_pt'])
B_data = len(event_B['data']['photon_pt'])
C_data = len(event_C['data']['photon_pt'])
D_data = len(event_D['data']['photon_pt'])
print('A_data',A_data)
print('B_data',B_data)
print('C_data',C_data)
print('D_data',D_data)

A_data 4103
B_data 4007
C_data 107
D_data 279


### calculate NA(nonprompt)

In [ ]:
import sympy as sp

# 定义变量
NA_prompt, NA, NA_bkg, R, NB, cB, NA_prompt, NB_bkg, NC, cC, ND, cD, NC_bkg, ND_bkg = sp.symbols('NA_prompt NA NA_bkg R NB cB NA_prompt NB_bkg NC cC ND cD NC_bkg ND_bkg')

# 根据公式【1】定义方程
equation = sp.Eq(NA_prompt, NA - NA_bkg - R * (NB - cB * NA_prompt - NB_bkg) * (NC - cC * NA_prompt - NC_bkg) / (ND - cD * NA_prompt - ND_bkg))

# 已知的值
values = {
    NA: 4103,  # A_data
    R: 1.157,  # 
    NB: 4007,  # B_data
    cB: 0.268,  # 
    NC: 107,  # C_data
    cC: 0.0035,  # 
    ND: 279,  # D_data
    cD: 0.0013,  # 
    NA_bkg: 263.0754951199856,
    NB_bkg: 64.41697392770189,  # B_dy
    NC_bkg: 3.5464861774302974,  # C_dy
    ND_bkg: 1.1569442372019945,  # D_dy
}
# 解方程
solution = sp.solve(equation.subs(values), NA_prompt)

# 输出解
print(f"NA_prompt = {solution[0]}")
print(f"NA_prompt = {solution[1]}")
NA_prompt = solution[0]


NA_prompt = 2540.00966888449
NA_prompt = 1090863.99669720


In [ ]:
N_nonprompt = A_data - A_bkg_prompt - NA_prompt
N_nonprompt

1299.91483599553

In [ ]:
import sympy as sp

# 定义变量
NA_prompt, NA, NA_bkg, R, NB, cB, NC, cC, ND, cD, NB_bkg, NC_bkg, ND_bkg = sp.symbols('NA_prompt NA NA_bkg R NB cB NC cC ND cD NB_bkg NC_bkg ND_bkg')

# 根据公式【1】定义方程
equation = sp.Eq(NA_prompt, NA - NA_bkg - R * (NB - cB * NA_prompt - NB_bkg) * (NC - cC * NA_prompt - NC_bkg) / (ND - cD * NA_prompt - ND_bkg))

# 已知的值
values = {
    NA: 4103,  # A_data
    R: 1.157,  # 
    NB: 4007,  # B_data
    cB: 0.268,  # 
    NC: 107,  # C_data
    cC: 0.0035,  # 
    ND: 279,  # D_data
    cD: 0.0013,  # 
    NA_bkg: 263.0754951199856,
    NB_bkg: 64.41697392770189,  # B_dy
    NC_bkg: 3.5464861774302974,  # C_dy
    ND_bkg: 1.1569442372019945,  # D_dy
}

# 解方程得到NA_prompt的值
NA_prompt_value = sp.solve(equation.subs(values), NA_prompt)[0]

# 计算公式【1】中每个变量的偏导数
partial_derivatives = {
    var: equation.lhs.diff(var) for var in [R, NB, cB, NC, cC, ND, cD]
}

# 计算每个变量的不确定度
uncertainties = {
    R: 0.2845 * values[R],
    NB: sp.sqrt(values[NB]),
    cB: sp.sqrt((sp.sqrt(NB)/NB)**2 + (sp.sqrt(NA)/NA)**2).subs(values),
    NC: sp.sqrt(values[NC]),
    cC: sp.sqrt((sp.sqrt(NC)/NC)**2 + (sp.sqrt(NC)/NC)**2).subs(values),
    ND: sp.sqrt(values[ND]),
    cD: sp.sqrt((sp.sqrt(ND)/ND)**2 + (sp.sqrt(ND)/ND)**2).subs(values),
}

# 使用误差传递公式计算NA_prompt的不确定度
sigma_NA_prompt_squared = sum([
    (partial_derivatives[var].subs(NA_prompt, NA_prompt_value) * uncertainties[var])**2 for var in partial_derivatives
])

sigma_NA_prompt = sp.sqrt(sigma_NA_prompt_squared)

# 使用公式【2】计算N_nonprompt的不确定度
sigma_N_nonprompt = sp.sqrt(values[NA] + values[NA_bkg] + sigma_NA_prompt**2)

print(f"NA_prompt uncertainty = {sigma_NA_prompt}")
print(f"N_nonprompt uncertainty = {sigma_N_nonprompt}")

# 解方程
solution = sp.solve(equation.subs(values), NA_prompt)

# 输出解
# print(f"NA_prompt = {solution[0]}")
# print(f"NA_prompt = {solution[1]}")
NA_prompt = solution[0]
N_nonprompt = A_data - A_bkg_prompt - NA_prompt
print("NA_nonprompt =",N_nonprompt)


NA_prompt uncertainty = 0
N_nonprompt uncertainty = 66.0762854216245
NA_nonprompt = 1299.91483599553


In [ ]:
import sympy as sp

# 定义变量
NA_prompt, NA, NA_bkg, R, NB, cB, NA_prompt, NB_bkg, NC, cC, ND, cD, NC_bkg, ND_bkg = sp.symbols('NA_prompt NA NA_bkg R NB cB NA_prompt NB_bkg NC cC ND cD NC_bkg ND_bkg')

# 根据公式定义方程
equation = sp.Eq(NA_prompt, NA - NA_bkg - R * (NB - cB * NA_prompt - NB_bkg) * (NC - cC * NA_prompt - NC_bkg) / (ND - cD * NA_prompt - ND_bkg))
N_nonprompt = NA - NA_bkg - NA_prompt

# 已知的值
values = {
    NA: 4103,  # A_data
    R: 1.157,  # 
    NB: 4007,  # B_data
    cB: 0.268,  # 
    NC: 107,  # C_data
    cC: 0.0035,  # 
    ND: 279,  # D_data
    cD: 0.0013,  # 
    NA_bkg: 263.0754951199856,
    NB_bkg: 64.41697392770189,  # B_dy
    NC_bkg: 3.5464861774302974,  # C_dy
    ND_bkg: 1.1569442372019945,  # D_dy
}

# 计算N_A, N_B, N_C, N_D的不确定性
sigma_NA = sp.sqrt(values[NA])
sigma_NB = sp.sqrt(values[NB])
sigma_NC = sp.sqrt(values[NC])
sigma_ND = sp.sqrt(values[ND])

# 使用误差传递公式计算N_nonprompt的不确定性
sigma_N_nonprompt = sp.sqrt((sp.diff(N_nonprompt, NA) * sigma_NA)**2 + (sp.diff(N_nonprompt, NB) * sigma_NB)**2 + (sp.diff(N_nonprompt, NC) * sigma_NC)**2 + (sp.diff(N_nonprompt, ND) * sigma_ND)**2)

# 使用已知的值求解
sigma_N_nonprompt_value = sigma_N_nonprompt.evalf(subs=values)

print(f"sigma_N_nonprompt = {sigma_N_nonprompt_value}")

# 解方程
solution = sp.solve(equation.subs(values), NA_prompt)

# 输出解
# print(f"NA_prompt = {solution[0]}")
# print(f"NA_prompt = {solution[1]}")
NA_prompt = solution[0]
N_nonprompt = A_data - A_bkg_prompt - NA_prompt
print("NA_nonprompt =",N_nonprompt)

sigma_N_nonprompt = 64.0546641549232
NA_nonprompt = 1299.91483599553


nonprompt 1128.0000000000032


# 废稿

In [ ]:
A_nonprompt = 1128
R = A_nonprompt * (D_data) / B_data / C_data
print('A_nonprompt',A_nonprompt)
print('B_data',B_data)
print('C_data',C_data)
print('D_data',D_data)
print('A/C',A_nonprompt/len(event_C['data']['photon_pt']))
print('B/D',len(event_B['data']['photon_pt'])/len(event_D['data']['photon_pt']))


print('R',R)

In [ ]:
A_nonprompt = 1128
R = A_nonprompt * (D_data-(A_data-A_nonprompt)*c_D) / (B_data-(A_data-A_nonprompt)*c_B) / (C_data-(A_data-A_nonprompt)*c_C)
print('A_nonprompt',A_nonprompt)
print('B_nonprompt',(B_data-(A_data-A_nonprompt)*c_B))
print('C_nonprompt',(C_data-(A_data-A_nonprompt)*c_C))
print('D_nonprompt',(D_data-(A_data-A_nonprompt)*c_D))


print('R',R)

In [ ]:
# R=1
qua_A = c_D - R*c_B*c_C
qua_B = R*B_data*c_C + R*C_data*c_B - D_data - A_data*c_D
qua_C = A_data*D_data- R*B_data*C_data
delta = qua_B*qua_B - 4*qua_A*qua_C
x1 = (-qua_B + np.sqrt(delta))/(2*qua_A)
x2 = (-qua_B - np.sqrt(delta))/(2*qua_A)
print('x1',x1)
print('x2',x2)

In [ ]:
nonprompt = A_data - x2
print('nonprompt',nonprompt)